In [0]:
import tensorflow as tf
import csv
import os
import glob
import h5py
import scipy.misc
import scipy.ndimage
import time
import random
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import io
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
from googleapiclient.discovery import build
from google.colab import files

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
!ls -l
#!zip -r dataset.zip DIV2K_train_HR.zip DIV2K_train_LR_bicubic_X4.zip DIV2K_valid_HR.zip DIV2K_valid_LR_bicubic_X4.zip Test.zip Train.zip

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls drive/

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
!rm -rf Train Test
!unzip Train.zip
!unzip Test.zip 

In [0]:
!ls

In [0]:
## Global Var.
g_epoch = 300
g_batch_size = 128
g_image_size = 33
g_label_size = 21
g_learn_rate = 1e-4 # default: 1e-4
g_color_dim = 3
g_scale = 3
g_checkpoint_dir = "checkpoint_srcnn"
g_output_dir = "output_srcnn"
g_extract_stride = 14
g_test_extract_stride = g_label_size
g_is_train = True

In [0]:
!rm -rf checkpoint_srcnn output_srcnn
!mkdir checkpoint_srcnn
!mkdir output_srcnn

In [0]:
!ls

In [0]:
g_counter = 0

In [0]:
### Network Model

# input and label images
images = tf.placeholder(tf.float32, [None, g_image_size, g_image_size, g_color_dim], name='images')
labels = tf.placeholder(tf.float32, [None, g_label_size, g_label_size, g_color_dim], name='labels')

# weights and biases
weights = {
    'w1': tf.Variable(tf.random_normal([9, 9, 3, 64], mean=0, stddev=1e-3), name='w1'),
    'w2': tf.Variable(tf.random_normal([1, 1, 64, 32], mean=0, stddev=1e-3), name='w2'),
    'w3': tf.Variable(tf.random_normal([5, 5, 32, 3], mean=0, stddev=1e-3), name='w3')
}

biases = {
    'b1': tf.Variable(tf.zeros([64]), name='b1'),
    'b2': tf.Variable(tf.zeros([32]), name='b2'),
    'b3': tf.Variable(tf.zeros([1]), name='b3')
}

# Layers
conv1 = tf.nn.relu(tf.nn.conv2d(images, weights['w1'], strides=[1, 1, 1, 1], padding='VALID') + biases['b1'])

conv2 = tf.nn.relu(tf.nn.conv2d(conv1, weights['w2'], strides=[1, 1, 1, 1], padding='VALID') + biases['b2'])

conv3 = tf.nn.conv2d(conv2, weights['w3'], strides=[1, 1, 1, 1], padding='VALID') + biases['b3']

# Output
pred = conv3

# Loss function
loss = tf.reduce_mean(tf.square(labels - pred))

# Saver
saver = tf.train.Saver()

In [0]:
tf.summary.scalar('loss', loss)
merged = tf.summary.merge_all()

In [0]:
!mkdir train_summary

In [0]:
train_writer = tf.summary.FileWriter('train_summary')
#test_writer = tf.summary.FileWriter('test_summary')

In [0]:
### Checkpointing functions - Save and Load

def save(sess, checkpoint_dir, scale, step):

    print("Saving checkpoint - step [{}]".format(step))
    model_name = "SRCNN.model"
    model_dir = "%s_%s_%s" % ("srcnn", "scale", scale)
    checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    saver.save(sess, os.path.join(checkpoint_dir, model_dir), global_step=step)


def load(sess, checkpoint_dir, scale):
    """
    Load the checkpoint.
    According to the scale, read different folder to load the models.
    """

    print(" [*] Reading checkpoints...")
    model_dir = "%s_%s_%s" % ("srcnn", "scale", scale)
    checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)

    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(checkpoint_dir, ckpt_name))
        return True
    else:
        return False

In [0]:
### Prep data from training

def modcrop(image, scale=3):
    """
    In order to scale down and up the original image, the first thing needed to
    do is to have no remainder while scaling operation.
    We need to find modulo of height (and width) and scale factor.
    Then, subtract the modulo from height (and width) of original image size.
    There would be no remainder even after scaling operation.
    """

    if len(image.shape) == 3:
        h, w, _ = image.shape
        h = h - np.mod(h, scale)
        w = w - np.mod(w, scale)
        image = image[0:h, 0:w, :]
    else:
        h, w = image.shape
        h = h - np.mod(h, scale)
        w = w - np.mod(w, scale)
        image = image[0:h, 0:w]

    return image
        

def input_setup(sess):
    preprocessed_folder = "preprocessed_scale_{}".format(g_scale)
    label_data_ext = "*_org.bmp"
    input_data_ext = "*_bicubic_scale_{}.bmp".format(g_scale)
    
    data_dir = os.path.join('Train', preprocessed_folder)
    label_data = glob.glob(os.path.join(data_dir, label_data_ext))
    input_data = glob.glob(os.path.join(data_dir, input_data_ext))
    
    label_data = sorted(label_data)
    input_data = sorted(input_data)
    
    sub_input_sequence = []
    sub_label_sequence = []
    
    padding = int(abs(g_image_size - g_label_size) / 2) # 6
    
    for i in range(len(input_data)):
        # Preprocess the input images
        #input_, label_ = preprocess(input_data[i], mean, stddev, g_scale)
        
        #input_ = modcrop(scipy.misc.imread(input_data[i], flatten=True, mode='YCbCr').astype(np.float64), g_scale) # / 255.
        #label_ = modcrop(scipy.misc.imread(label_data[i], flatten=True, mode='YCbCr').astype(np.float64), g_scale) # / 255.
        
        input_ = modcrop(scipy.misc.imread(input_data[i], mode='RGB'), g_scale) # / 255.
        label_ = modcrop(scipy.misc.imread(label_data[i], mode='RGB'), g_scale) # / 255.
        
        #plt.imshow(label_)
        
        input_ = input_ / 255
        label_ = label_ / 255
        
        #plt.imshow(label_)
        
        #input_ *= 255
        #label_ *= 255
        
        if len(input_.shape) == 3:
            h, w, _ = input_.shape
        else:
            h, w = input_.shape

        # Crop the input images
        for x in range(0, h-g_image_size+1, g_extract_stride):
            for y in range(0, w-g_image_size+1, g_extract_stride):
                sub_input = input_[x:x+g_image_size, y:y+g_image_size] # [33 x 33]
                sub_label = label_[x+padding:x+padding+g_label_size, y+padding:y+padding+g_label_size] # [21 x 21]

                #plt.imshow(sub_label)
                
                # Make channel value
                sub_input = sub_input.reshape([g_image_size, g_image_size, g_color_dim])
                sub_label = sub_label.reshape([g_label_size, g_label_size, g_color_dim])

                #plt.imshow(sub_label)
                
                sub_input_sequence.append(sub_input)
                sub_label_sequence.append(sub_label)
    
    arrdata = np.asarray(sub_input_sequence) # [?, 33, 33, 1]
    arrlabel = np.asarray(sub_label_sequence) # [?, 21, 21, 1]

    savepath = 'checkpoint_srcnn/train.h5'
    with h5py.File(savepath, 'w') as hf:
        hf.create_dataset('data', data=arrdata)
        hf.create_dataset('label', data=arrlabel)


def read_data(path):
    with h5py.File(path, 'r') as hf:
        data = np.array(hf.get('data'))
        label = np.array(hf.get('label'))

    return data, label

In [0]:
### Training
def train_runner(g_counter):
    with tf.Session() as sess:
        input_setup(sess)
        print("input done.")
        data_dir = os.path.join('./{}'.format(g_checkpoint_dir), "train.h5")
        train_data, train_label = read_data(data_dir)

        ###
        '''
        var_list_1 = [weights['w1'], weights['w2'], biases['b1'], biases['b2']]
        var_list_2 = [weights['w3'], biases['b3']]
        
        train_op_1 = tf.train.AdamOptimizer(g_learn_rate).minimize(loss, var_list=var_list_1)
        train_op_2 = tf.train.AdamOptimizer(g_learn_rate/10.).minimize(loss, var_list=var_list_2)
        
        train_op = tf.group(train_op_1, train_op_2)
        '''
        
        ###
        #'''
        var_list__1 = [weights['w1'], weights['w2'], biases['b1'], biases['b2']]
        var_list__2 = [weights['w3'], biases['b3']]
        
        opt_1 = tf.train.AdamOptimizer(g_learn_rate)
        opt_2 = tf.train.AdamOptimizer(g_learn_rate/10.)
        
        grads = tf.gradients(loss, var_list__1 + var_list__2)
        grads_1 = grads[:len(var_list__1)]
        grads_2 = grads[len(var_list__1):]
        
        train_op__1 = opt_1.apply_gradients(zip(grads_1, var_list__1))
        train_op__2 = opt_2.apply_gradients(zip(grads_2, var_list__2))
        
        train_op = tf.group(train_op__1, train_op__2)
        #'''
        
        ###
        #train_op = tf.train.AdamOptimizer(g_learn_rate).minimize(loss)
        
        
        sess.run(tf.global_variables_initializer())

        iter_counter = 0
        avg_loss = 0
        avg_50_loss = 0
        start_time = time.time()

        if load(sess, g_checkpoint_dir, g_scale):
            print('Checkpoint load SUCCESS.')
        else:
            print('Checkpoint load FAILED.')

        for ep in range(g_epoch):
            batch_idxs = len(train_data) // g_batch_size
            print(len(train_data) ,batch_idxs)
            print(g_batch_size)
            shuffled_data = list(zip(train_data, train_label))
            random.shuffle(shuffled_data)
            train_data, train_label = zip(*shuffled_data)

            for idx in range(0, batch_idxs):
                iter_counter += 1
                batch_images = train_data[idx*g_batch_size : (idx+1)*g_batch_size]
                batch_labels = train_label[idx*g_batch_size : (idx+1)*g_batch_size]

                _, err, summary = sess.run([train_op, loss, merged], feed_dict={images: batch_images, labels: batch_labels})
                g_counter += 1
                avg_loss += err
                avg_50_loss += err

                train_writer.add_summary(summary, ep)
                
                if iter_counter % 10 == 0:
                    print("Epoch: [%2d], step: [%2d], time: [%4.4f], loss: [%.8f]" % ((ep+1), iter_counter, time.time()-start_time, err))
                if iter_counter % 50 == 0:
                    save(sess, g_checkpoint_dir, g_scale, iter_counter)
                    print("==> Epoch: [%2d], avg. loss of 50 steps: [%.8f], avg. loss: [%.8f]" % ((ep+1), avg_50_loss/50, avg_loss/iter_counter))
                    avg_50_loss = 0
                if g_counter % 100 == 0:
                    test_runner(sess, g_counter)

        save(sess, g_checkpoint_dir, g_scale, iter_counter)
        print("==> Epoch: [%2d], avg. loss of 50 steps: [%.8f], avg. loss: [%.8f]" % ((ep+1), avg_50_loss/50, avg_loss/iter_counter))
        avg_50_loss = 0
#train_runner()

In [0]:
### Prep for testing

def input_setup_test(sess, img_index):
    preprocessed_folder = "/preprocessed_scale_{}".format(g_scale)
    label_data_ext = "*_org.bmp"
    input_data_ext = "*_bicubic_scale_{}.bmp".format(g_scale)
    
    data_dir = os.path.join(os.sep, (os.path.join(os.getcwd(), 'Test')), "Set5" + preprocessed_folder)
    label_data = glob.glob(os.path.join(data_dir, label_data_ext))
    input_data = glob.glob(os.path.join(data_dir, input_data_ext))
    
    input_data = sorted(input_data)
    label_data = sorted(label_data)
    
    sub_input_sequence = []
    sub_label_sequence = []
    
    padding = int(abs(g_image_size - g_label_size) / 2) # 6
    
    #input_ = modcrop(scipy.misc.imread(input_data[img_index], flatten=True, mode='YCbCr').astype(np.float64), g_scale) # / 255.
    #label_ = modcrop(scipy.misc.imread(label_data[img_index], flatten=True, mode='YCbCr').astype(np.float64), g_scale) # / 255.
    
    input_ = modcrop(scipy.misc.imread(input_data[img_index], mode='RGB'), g_scale) # / 255.
    label_ = modcrop(scipy.misc.imread(label_data[img_index], mode='RGB'), g_scale) # / 255.
    
    input_ = input_ / 255
    label_ = label_ / 255
    
    #input_ *= 255
    #label_ *= 255

    if len(input_.shape) == 3:
        h, w, _ = input_.shape
    else:
        h, w = input_.shape

    # Crop the input images
    # Numbers of sub-images in height and width of image are needed to compute merge operation.
    nx = ny = 0
    for x in range(0, h-g_image_size+1, g_test_extract_stride):
        nx += 1; ny = 0
        for y in range(0, w-g_image_size+1, g_test_extract_stride):
            ny += 1
            sub_input = input_[x:x+g_image_size, y:y+g_image_size] # [33 x 33]
            sub_label = label_[x+padding:x+padding+g_label_size, y+padding:y+padding+g_label_size] # [21 x 21]

            sub_input = sub_input.reshape([g_image_size, g_image_size, g_color_dim])
            sub_label = sub_label.reshape([g_label_size, g_label_size, g_color_dim])

            sub_input_sequence.append(sub_input)
            sub_label_sequence.append(sub_label)
    
    print("Saving the original images... size: [{} x {}]".format(g_test_extract_stride*nx, g_test_extract_stride*ny))
    image_path = os.path.join(os.getcwd(), g_output_dir)
    org_img_path = os.path.join(image_path, "{}_test_org_img.png".format(img_index))
    bicubic_img_path = os.path.join(image_path, "{}_test_bicubic_img.png".format(img_index))
    imsave(label_[padding:padding+g_test_extract_stride*nx, padding:padding+g_test_extract_stride*ny], org_img_path)
    imsave(input_[padding:padding+g_test_extract_stride*nx, padding:padding+g_test_extract_stride*ny], bicubic_img_path)
    
    arrdata = np.asarray(sub_input_sequence) # [?, 33, 33, 1]
    arrlabel = np.asarray(sub_label_sequence) # [?, 21, 21, 1]

    savepath = os.path.join(os.getcwd(), 'checkpoint_srcnn/test.h5')
    with h5py.File(savepath, 'w') as hf:
        hf.create_dataset('data', data=arrdata)
        hf.create_dataset('label', data=arrlabel)
    
    return nx, ny


def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h*size[0], w*size[1], 1))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img


def psnr(img1, img2):
    mse = np.square(img1 - img2)
    mse = mse.mean()
    psnr = 20*np.log10(255./(np.sqrt(mse)))
    return psnr


def imsave(image, path):
    image = image*255
    #output_image = scipy.misc.toimage(image, high=np.max(image), low=np.min(image), mode='L')
    #output_image = scipy.misc.toimage(image, high=255, low=np.min(image), mode='L')
    output_image = scipy.misc.toimage(image, high=255, low=0)
    output_image.save(path)

In [0]:
### Testing

def test_runner(sess, g_counter):
    psnr_srcnn = []
    psnr_bicubic = []
    counter_arr = []
    psnr_csv_row = []
    
    #with tf.Session() as sess:
    #if load(sess, g_checkpoint_dir, g_scale):
    #    print("Checkpoint load SUCCESS.")
    #else:
    #    print("Checkpoint load FAILED.")

    for i in range(5):
        nx, ny = input_setup_test(sess, i)
        data_dir = os.path.join('./{}'.format(g_checkpoint_dir), "test.h5")
        test_data, test_label = read_data(data_dir)
        result = pred.eval({images: test_data, labels: test_label})
        #print(np.array(result).shape)
        n, w, h, _ = np.array(result).shape
        result_0 = merge(np.reshape(result[:,:,:,0], [n,w,h,1]), [nx, ny])
        result_1 = merge(np.reshape(result[:,:,:,1], [n,w,h,1]), [nx, ny])
        result_2 = merge(np.reshape(result[:,:,:,2], [n,w,h,1]), [nx, ny])
        #print(result_0.shape)
        #print(result_1.shape)
        #print(result_2.shape)
        #result = result_0.squeeze()
        #result = np.dstack((result_0.squeeze(), result_1.squeeze(), result_2.squeeze()))
        w, h = result_0.squeeze().shape
        #print(w, h)
        result = np.zeros([w, h, 3])
        result[:,:,0] = result_0.squeeze()
        result[:,:,1] = result_1.squeeze()
        result[:,:,2] = result_2.squeeze()
        
        #plt.imshow(result)
        #plt.pause(0.05)

        # Save output image
        output_path = os.path.join(os.getcwd(), g_output_dir)
        image_path = os.path.join(output_path, "{}_test_img.png".format(i))
        imsave(result, image_path)

        # PSNR
        label_path = os.path.join(output_path, "{}_test_org_img.png".format(i))
        bicubic_path = os.path.join(output_path, "{}_test_bicubic_img.png".format(i))

        #bicubic_img = scipy.misc.imread(bicubic_path, flatten=True, mode='YCbCr').astype(np.float64)
        bicubic_img = scipy.misc.imread(bicubic_path, mode='RGB')
        #plt.imshow(bicubic_img)
        #plt.pause(0.05)
        
        #label_img = scipy.misc.imread(label_path, flatten=True, mode='YCbCr').astype(np.float64)
        label_img = scipy.misc.imread(label_path, mode='RGB')
        #plt.imshow(label_img)
        #plt.pause(0.05)
        
        #output_img = scipy.misc.imread(image_path, flatten=True, mode='YCbCr').astype(np.float64)
        output_img = scipy.misc.imread(image_path, mode='RGB')
        #plt.imshow(output_img)
        #plt.pause(0.05)
        
        bicubic_psnr_value = psnr(label_img, bicubic_img)
        srcnn_psnr_value = psnr(label_img, output_img)

        print("Image {}: Bicubic PSNR: [{}]".format(i+1, bicubic_psnr_value))
        print("Image {}: SRCNN PSNR: [{}]".format(i+1, srcnn_psnr_value))

        psnr_srcnn.append(srcnn_psnr_value)
        psnr_bicubic.append(bicubic_psnr_value)
        counter_arr = [g_counter]
    psnr_csv_row = counter_arr + psnr_srcnn + psnr_bicubic
    avg = 0
    for psnr_val in psnr_bicubic:
        avg += psnr_val
    avg /= len(psnr_bicubic)
    print("Avg. PSNR Bicubic: [{}]".format(avg))
    
    avg = 0
    for psnr_val in psnr_srcnn:
        avg += psnr_val
    avg /= len(psnr_srcnn)
    print("Avg. PSNR SRCNN: [{}]".format(avg))
    
    csv_path = os.path.join(output_path, 'set5_psnr_validation.csv')
    with open(csv_path, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(psnr_csv_row)
#test_runner()

In [0]:
train_runner(g_counter)
g_counter += 170*g_epoch

In [0]:
#g_counter=0 68340
print(g_counter)

In [0]:
!ls train_summary

In [0]:
#!zip -r ckpt_3dim_col_rev_51000.zip checkpoint_srcnn
!zip -r op_col_6800.zip output_srcnn
#files.download('ckpt_3dim_col_rev_51000.zip')
files.download('op_col_6800.zip')

In [0]:
!cat output/set5_psnr_validation.csv

In [0]:
#files.download('output_srcnn/set5_psnr_validation.csv')
files.download('ckpt_3dim_col_rev_34000.zip')

In [0]:
!rm -rf checkpoint_srcnn
!mkdir checkpoint_srcnn
!rm -rf output_srcnn
!mkdir output_srcnn

In [0]:
!zip -r output_ckpt_srcnn_f.zip output_srcnn checkpoint_srcnn

In [0]:
!ls


In [0]:
files.download('output_ckpt_srcnn_f.zip')

In [0]:
!rm -rf output.zip

In [0]:
!zip -r train1.zip Train/preprocessed_scale_3
files.download('train1.zip')


## Image Preprocess code

In [0]:
!pip install scikit-image

In [0]:
import os
import glob
import h5py
import scipy.misc
import scipy.ndimage
import numpy as np
from scipy import misc
import skimage.transform as transform

In [0]:
def normalize_1(image):
    image = image/image.max()
    return image

In [0]:
def modcrop_1(image, scale=3):
    if len(image.shape) == 3:
        h, w, _ = image.shape
        h = h - np.mod(h, scale)
        w = w - np.mod(w, scale)
        image = image[0:h, 0:w, :]
    else:
        h, w = image.shape
        h = h - np.mod(h, scale)
        w = w - np.mod(w, scale)
        image = image[0:h, 0:w]

    return image  

In [0]:
def imread_1(path, is_grayscale=True):
    if is_grayscale:
        return scipy.misc.imread(path, flatten=True, mode='YCbCr').astype(np.float64)
    else:
        return scipy.misc.imread(path, mode='RGB')

In [0]:
def preprocess_1(path, mean=0, stddev=1e-3, scale=3):
    # Read Image
    input_ = imread_1(path, is_grayscale=False)

    # Crop the suitable size to fit the scale
    input_modcropped = modcrop_1(input_, scale)

    # Normalization
    label_ = normalize_1(input_modcropped)
    #label_ = input_modcropped / 255

    #input_ = scipy.ndimage.interpolation.zoom(label_, (1./scale), prefilter=False)
    #input_ = scipy.ndimage.interpolation.zoom(input_, (scale/1.), prefilter=False)

    #input_ = misc.imresize(label_, 1./scale, 'bicubic')  ### before
    #input_ = misc.imresize(input_, float(scale), 'bicubic')
    
    input_ = transform.rescale(label_, 1./scale)
    input_ = transform.rescale(input_, float(scale))
    
    return input_, label_

In [0]:
def imsave_1(image, path):
    image = image*255
    output_image = misc.toimage(image, high=np.max(image), low=np.min(image))
    output_image.save(path)

In [0]:
!mkdir Test/Set5/preprocessed_scale_2
!mkdir Test/Set5/preprocessed_scale_4
!ls

In [0]:
sorted(os.listdir('Test/Set5'))

In [0]:
scale = 3
img_dir = 'Test/Set5'
image_list = list(filter(lambda x: x[-3:] == 'bmp', os.listdir(img_dir)))
preprocessed_dir = list(filter(lambda x: x[-3:] != 'bmp', sorted(os.listdir(img_dir))))[scale-2] # 0->2; 1->3; 2->4


for image in image_list:
    #print(image)
    input_, label_ = preprocess_1(os.path.join(img_dir, image), 0.5, 1e-4, scale)
    filename, ext = image.split('.')
    imsave_1(label_, './{}/{}_org.{}'.format(os.path.join(img_dir, preprocessed_dir), filename, ext))
    imsave_1(input_, './{}/{}_bicubic_scale_{}.{}'.format(os.path.join(img_dir, preprocessed_dir), filename, scale, ext))

In [0]:
!ls Train/preprocessed_scale_3

In [0]:
!ls Test/Set5

In [0]:
!cp Test/Set5/baby_GT.bmp Test/Set5/preprocessed_scale_3/baby_GT_col.bmp
!cp Test/Set5/bird_GT.bmp Test/Set5/preprocessed_scale_3/bird_GT_col.bmp
!cp Test/Set5/butterfly_GT.bmp Test/Set5/preprocessed_scale_3/butterfly_GT_col.bmp
!cp Test/Set5/head_GT.bmp Test/Set5/preprocessed_scale_3/head_GT_col.bmp
!cp Test/Set5/woman_GT.bmp Test/Set5/preprocessed_scale_3/woman_GT_col.bmp

In [0]:
!ls Test/Set5/preprocessed_scale_2/

In [0]:
!zip -r t3.zip Test/Set5/preprocessed_scale_3
files.download('t3.zip')

In [0]:
files.download('pre3.zip')

### OpenCV image downsampler

In [0]:
!wget https://raw.githubusercontent.com/wqi/img-downsampler/master/downsample.py

# View TF Graph

In [0]:
from IPython.display import clear_output, Image, display, HTML
import tensorflow as tf
import numpy as np
from google.colab import files

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
"""Create a sample tensor"""
#sample_placeholder= tf.placeholder(dtype=tf.float32) 
"""Show it"""
graph_def = tf.get_default_graph().as_graph_def()
show_graph(graph_def)

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
# Upload the file to Drive. See:
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'dataset.zip',
  'mimeType': 'application/zip'
}
media = MediaFileUpload('dataset.zip', 
                        mimetype='application/zip',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

In [0]:
#### dataset.zip File ID: 1IstPsPA0mFfgdOMn6XLAMKy0unZFpOFT

In [0]:
!cp -r /train_summary ./train_summary

In [0]:
!ls

In [0]:
LOG_DIR = 'train_summary'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
! curl http://localhost:6006

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1
! unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"